In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

### Load Data

In [4]:
# Load the data as numpy arrays
img_1 = pd.read_csv('./../data/feaSubEImg_1.csv', header=None).to_numpy()
img_2 = pd.read_csv('./../data/feaSubEImg_2.csv', header=None).to_numpy()
overt_1 = pd.read_csv('./../data/feaSubEOvert_1.csv', header=None).to_numpy()
overt_2 = pd.read_csv('./../data/feaSubEOvert_2.csv', header=None).to_numpy()

In [5]:
# Combine the features
img_X = np.hstack((img_1, img_2)).T
img_y = np.array([0] * img_1.shape[1] + [1] * img_2.shape[1])

overt_X = np.hstack((overt_1, overt_2)).T
overt_y = np.array([0] * overt_1.shape[1] + [1] * overt_2.shape[1])

### Cross-Train function

In [ ]:
def cross_train_test(train_X, train_y, test_X, test_y, C_value, title_prefix=""):
    # Train SVM
    model = SVC(kernel='linear', C=C_value)
    model.fit(train_X, train_y)

    # Predict and get decision function
    y_pred = model.predict(test_X)
    decision_scores = model.decision_function(test_X)

    # Accuracy
    acc = accuracy_score(test_y, y_pred)

    # Plot ROC
    plot_roc_curve(test_y, decision_scores, title=f"{title_prefix} ROC Curve")

    return acc